In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import matplotlib as mpl
from datetime import datetime
import plotly
plotly.tools.set_credentials_file(username='bzjin', api_key='jmaLQFWVMgugFAITZZBC')
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
restaurants = pd.read_csv("src/assets/data/restaurant_categories.csv")
restaurants.head()

,id,name,image_url,name_lower,url,review_count,categories,rating,coordinates__latitude,coordinates__longitude,...,location__city,location__zip_code,location__country,location__state,location__display_address__001,location__display_address__002,location__display_address__003,phone,display_phone,distance
0,wentworth-seafood-house-chicago-3,Wentworth_Seafood_House,https://s3-media4.fl.yelpcdn.com/bphoto/aQRbuF...,wentworth_seafood_house,https://www.yelp.com/biz/wentworth-seafood-hou...,7,"bubbletea,seafood,cantonese,",4.0,41.85230128,-87.63184867,...,Chicago,60616.0,US,IL,2229_S_Wentworth_Ave,"Chicago,_IL_60616",NaN,1.312808e+10,(312)_808-0899,739.992474
1,mccb-chicago-时尚食谱-chicago,MCCB_Chicago_时尚食谱,https://s3-media3.fl.yelpcdn.com/bphoto/Jnmu_r...,mccb_chicago,https://www.yelp.com/biz/mccb-chicago-%E6%97%B...,60,"chinese,",4.5,41.8539,-87.63365,...,Chicago,60616.0,US,IL,2138_South_Archer_Ave,"Chicago,_IL_60616",NaN,1.312881e+10,(312)_881-0168,933.643538
2,qing-xiang-yuan-dumplings-chicago-3,Qing_Xiang_Yuan_Dumplings,https://s3-media3.fl.yelpcdn.com/bphoto/d0I1bE...,qing_xiang_yuan_dumplings,https://www.yelp.com/biz/qing-xiang-yuan-dumpl...,488,"chinese,bbq,",4.0,41.85505,-87.63219,...,Chicago,60616.0,US,IL,2002_S_Wentworth_Ave,Ste_103,"Chicago,_IL_60616",1.312799e+10,(312)_799-1118,1045.817120
3,mins-noodle-house-chicago-15,Mins_Noodle_House,https://s3-media4.fl.yelpcdn.com/bphoto/J9kSN1...,mins_noodle_house,https://www.yelp.com/biz/mins-noodle-house-chi...,165,"noodles,comfortfood,szechuan,",4.5,41.8353899,-87.64586,...,Chicago,60608.0,US,IL,3235_S_Halsted_St,"Chicago,_IL_60608",NaN,1.312465e+10,(312)_465-2999,1650.001668
4,minghin-cuisine-chicago-3,MingHin_Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/tJlRSu...,minghin_cuisine,https://www.yelp.com/biz/minghin-cuisine-chica...,1709,"seafood,dimsum,cantonese,",4.0,41.85325643,-87.63490268,...,Chicago,60616.0,US,IL,2168_S_Archer_Ave,"Chicago,_IL_60616",NaN,1.312808e+10,(312)_808-1999,889.326821


In [108]:
namesSeries = pd.Series(restaurants["name_lower"])
namesArray = np.array(namesSeries, dtype=pd.Series)

categorySeries = pd.Series(restaurants["categories"])
categoryArray = np.array(categorySeries, dtype=pd.Series)

#A list of all the categories
unique_categories = []
category_count = {}

for x in categoryArray:
    str_list = [y.strip() for y in x.split(',')][:-1]
    for el in str_list:
        if not el in unique_categories:
            unique_categories.append(el)
            category_count[el] = 0

for x in categoryArray:
    str_list = [y.strip() for y in x.split(',')][:-1]
    for el in str_list:
        category_count[el] = category_count[el] + 1

#for x in category_count:
    #print(x,",",category_count[x])

#Empy count of categories dict
category_phrases = {}
category_words = {}
category_bigrams = {}
category_associations = {}
all_words = {}
all_bigrams = {}
shared_food = {}
single = {}
double = {}
triple = {}
all_counts = {}

for x in unique_categories:
    category_phrases[x] = []
    category_words[x] = []
    category_bigrams[x] = []

#Dictionary of {names: categories array}
names_categories = {}
for x in namesArray:
    str_list = categoryArray[np.where(namesArray==x)][0]
    array_list = [y.strip() for y in str_list.split(',')][:-1]
    names_categories[x] = array_list

    for category in array_list:
        if not category in all_counts:
            all_counts[category] = 1
        else:
            all_counts[category] += 1
            
        if len(array_list) == 1:
            if not category in category_associations:
                category_associations[category] = 1
            else:
                category_associations[category] += 1
                
        elif len(array_list) == 2:
            option1 = str(array_list[0] + "," + array_list[1])
            option2 = str(array_list[1] + "," + array_list[0])
            if (not option1 in category_associations) and (not option2 in category_associations):
                category_associations[option1] = 1
            elif (option1 in category_associations):
                category_associations[option1] += 1
            else:
                category_associations[option2] += 1
                
        elif len(array_list) == 3:
            option1 = str(array_list[0] + "," + array_list[1])
            option2 = str(array_list[1] + "," + array_list[0])
            option3 = str(array_list[0] + "," + array_list[2])
            option4 = str(array_list[2] + "," + array_list[0])
            option5 = str(array_list[2] + "," + array_list[1])
            option6 = str(array_list[1] + "," + array_list[2])
            if (not option1 in category_associations) and (not option2 in category_associations):
                category_associations[option1] = 1
            elif (option1 in category_associations):
                category_associations[option1] += 1
            else:
                category_associations[option2] += 1
                
            if (not option3 in category_associations) and (not option4 in category_associations):
                category_associations[option3] = 1
            elif (option3 in category_associations):
                category_associations[option3] += 1
            else:
                category_associations[option4] += 1
                
            if (not option5 in category_associations) and (not option6 in category_associations):
                category_associations[option5] = 1
            elif (option5 in category_associations):
                category_associations[option5] += 1
            else:
                category_associations[option6] += 1
                
#for x in all_counts:
   # print('"', x, '":', all_counts[x], ",")

In [109]:
#Empy count of categories dict
single = {}
double = {}
triple = {}

#Dictionary of {names: categories array}
names_categories = {}
for x in namesArray:
    str_list = categoryArray[np.where(namesArray==x)][0]
    array_list = [y.strip() for y in str_list.split(',')][:-1]
    names_categories[x] = array_list

    for category in array_list:
        if len(array_list) == 1:
            if not category in single:
                single[category] = 1
            else:
                single[category] += 1
                
        elif len(array_list) == 2:
            option1 = str(array_list[0] + "," + array_list[1])
            option2 = str(array_list[1] + "," + array_list[0])
            if (not option1 in double) and (not option2 in double):
                double[option1] = 1
            elif (option1 in double):
                double[option1] += 1
            else:
                double[option2] += 1
                
        elif len(array_list) == 3:
            o1 = str(array_list[0] + "," + array_list[1] + "," + array_list[2])
            o2 = str(array_list[0] + "," + array_list[2] + "," + array_list[1])
            o3 = str(array_list[1] + "," + array_list[0] + "," + array_list[2])
            o4 = str(array_list[1] + "," + array_list[2] + "," + array_list[0])
            o5 = str(array_list[2] + "," + array_list[0] + "," + array_list[1])
            o6 = str(array_list[2] + "," + array_list[1] + "," + array_list[0])
            options = [o1, o2, o3, o4, o5, o6]
            
            marker = ""
            for o in options:
                if o in triple:
                    marker = o
                    triple[o] += 1
            
            if marker == "":
                triple[o1] = 1

#for x in triple:
    #print(x, ",", triple[x])

In [110]:
### Associations - most common pairings
for key, value in category_associations.items():
    unique, counts = np.unique(value, return_counts=True)
    association_counter = {}
    
    for x in unique:
        association_counter[x] = counts[np.where(unique==x)][0]
    
    popular_associations = sorted(association_counter, key = association_counter.get, reverse = True)
    top_associations = popular_associations[:20]
    for association in top_associations:
        print(key, "," , association)

bubbletea,seafood , 3
bubbletea,cantonese , 3
cantonese,seafood , 14
chinese , 40
chinese,bbq , 9
noodles,comfortfood , 3
noodles,szechuan , 3
szechuan,comfortfood , 3
seafood,dimsum , 14
cantonese,dimsum , 13
caribbean,seafood , 3
caribbean,vegetarian , 3
vegetarian,seafood , 3
korean,bbq , 4
chinese,noodles , 5
mongolian,hotpot , 6
mongolian,chinese , 3
chinese,hotpot , 8
indpak,italian , 3
indpak,cajun , 3
cajun,italian , 3
chinese,seafood , 7
chinese,asianfusion , 9
asianfusion,seafood , 3
italian,pizza , 26
italian,sandwiches , 3
sandwiches,pizza , 6
chicken_wings,korean , 3
chicken_wings,salad , 3
salad,korean , 3
ramen , 2
pizza , 9
cajun,seafood , 2
italian , 1
british,pubs , 3
british,breakfast_brunch , 3
breakfast_brunch,pubs , 6
vietnamese , 1
delis,grocery , 6
delis,sandwiches , 6
sandwiches,grocery , 3
taiwanese,shavedice , 3
taiwanese,bubbletea , 5
bubbletea,shavedice , 3
noodles,panasian , 2
burgers,mexican , 3
burgers,cocktailbars , 3
cocktailbars,mexican , 3
szechuan ,

In [111]:
#Array of every menu item in each category. 
#Category_phrases has full item name
#Category_words has words that make up item names
prep = ['with', 'and', 'w/', 'the', 'a', 'an', '&', 'in', 'to', 'of']
for key, value in names_categories.items():
    my_file = Path("src/assets/data/menus/" + key + ".csv")
   
    if my_file.is_file():
        current_menu = pd.read_csv("src/assets/data/menus/" + key + ".csv")
        menuSeries = pd.Series(current_menu["name"])
        menuArray = np.array(menuSeries, dtype=pd.Series)

        for x in value:
            already_counted = []
            for phrase in menuArray:
                stripped = re.sub(r'([^\s\w]|_|[0-9])+', '', phrase)
                stripped = stripped.lower().rstrip().lstrip()
                category_phrases[x].append(stripped)
                                
                words = phrase.lower().split()
                for word in words:
                    if not bool(re.search(r'\d', word)):
                        if not words.index(word) == len(words) - 1:
                            category_bigrams[x].append(word + " " + words[words.index(word)+1])
                            bigram = word + " " + words[words.index(word)+1]
                            if not bigram in all_bigrams:
                                all_bigrams[bigram] = 1
                            else:
                                all_bigrams[bigram] += 1
                                
                            #shared food
                            if not bigram in shared_food: #{"fried rice": ["chinese", "chinese", "cantonese"...]}
                                already_counted.append(bigram) #[fried rice, pork chops, fish fillet...]
                                shared_food[bigram] = []
                                shared_food[bigram].append(x)
                            elif (not bigram in already_counted) and (bigram in shared_food):
                                shared_food[bigram].append(x)
                                already_counted.append(bigram)
                        if not word in prep:
                            category_words[x].append(word)
                        if not word in all_words:
                            all_words[word] = 1
                        else:
                            all_words[word] += 1
                        

#for x in all_words:
    #print(x, ",", all_words[x])

#for x in all_bigrams:
    #print(x, ",", all_bigrams[x])


In [112]:
### Phrases - most common in each cuisine
for key, value in category_phrases.items():
    unique, counts = np.unique(value, return_counts=True)
    phrase_counter = {}
    
    for x in unique:
        phrase_counter[x] = counts[np.where(unique==x)]
    
    popular_phrases = sorted(phrase_counter, key = phrase_counter.get, reverse = True)
    top_phrases = popular_phrases[:20]
    for phrase in top_phrases:
        print(key, "," , phrase, ",", counts[np.where(unique==phrase)][0])


bubbletea , braised chicken leg rice box , 2
bubbletea , braised pork belly rice box , 2
bubbletea , braised pork over rice , 2
bubbletea , crispy bbq pork rice box , 2
bubbletea , crispy spiced chicken with chili sauce rice box , 2
bubbletea , piece homemade potstickers , 2
bubbletea , popcorn chicken , 2
bubbletea , spicy beef and tendon noodle soup , 2
bubbletea , spicy beef noodle soup , 2
bubbletea , squid in thick broth , 2
bubbletea , taipei milk tea , 2
bubbletea , ba wan , 1
bubbletea , basil chicken , 1
bubbletea , basil fried squid with basil , 1
bubbletea , basil king mushroom , 1
bubbletea , basil tofu , 1
bubbletea , berries and lime fruit tea , 1
bubbletea , black forest latte , 1
bubbletea , black forest mousse , 1
bubbletea , black tea latte , 1
seafood , crab rangoon , 7
seafood , salt and pepper shrimp , 7
seafood , wonton soup , 7
seafood , egg drop soup , 6
seafood , mongolian beef , 6
seafood , bbq pork fried rice , 5
seafood , chicken fried rice , 5
seafood , hot

british , deluxe gravy chips , 1
british , grapefruit juice , 1
british , gravy sauce , 1
british , hand cut chips , 1
british , kombuchade , 1
british , large drip coffee , 1
british , mash , 1
british , mushroom and kale pie , 1
british , orange juice , 1
british , peas , 1
british , pickled egg , 1
british , pleasant farms salad , 1
pubs , anchors away fish fry , 1
pubs , bacon wrapped potatoes , 1
pubs , bangers and mash , 1
pubs , blackened tuna sandwich , 1
pubs , boneless tenders , 1
pubs , breakfast poutine , 1
pubs , bubble cake , 1
pubs , buffalo chicken wrap , 1
pubs , build your own burger , 1
pubs , build your own pizza , 1
pubs , caesar salad , 1
pubs , california dreaming pizza , 1
pubs , chicago philly sandwich , 1
pubs , chicken balti pie , 1
pubs , chicken risotto soup , 1
pubs , chicken salad , 1
pubs , chicken wings , 1
pubs , chili , 1
pubs , cold pork pie , 1
pubs , coriander chutney sauce , 1
breakfast_brunch ,  , 41
breakfast_brunch , avocado fries , 2
breakfast

senegalese , attieke , 1
senegalese , baobab , 1
senegalese , bissap sorrel , 1
senegalese , bottled water , 1
senegalese , brochette chicken , 1
senegalese , caramel cake , 1
senegalese , coconut beverage , 1
senegalese , couscous , 1
senegalese , dibi chicken  pcs , 1
senegalese , dibi chicken  pcs lunch special , 1
senegalese , dibi lamb , 1
senegalese , fataya , 1
senegalese , french fries , 1
senegalese , fried plantain , 1
senegalese , ginger , 1
senegalese , grilled fish , 1
senegalese , honeydew and pineapple , 1
senegalese , house thiebaut ga naar , 1
senegalese , house thiebaut jun , 1
senegalese , jollof rice , 1
spanish , aceitunas aliñadas , 1
spanish , alcachofas marinada , 1
spanish , banana foster , 1
spanish , berenjena con setas , 1
spanish , cabrales , 1
spanish , calamares a la plancha , 1
spanish , calamares en su tinta , 1
spanish , caldero madrileño , 1
spanish , cazuelita de chocolate , 1
spanish , cazuelita de pollo , 1
spanish , champiñones rellenos , 1
spanis

mediterranean , grilled chicken shawarma platter , 2
mediterranean , grilled chicken shawarma wrap , 2
mediterranean , herb fries with kalamata sauce , 2
mediterranean , hummus , 2
mediterranean , a la turca lentil soup , 1
mediterranean , arabian night dinner , 1
mediterranean , arabic mint tea dinner , 1
mediterranean , assortment of hot tea dinner , 1
mediterranean , baba gannoush , 1
mediterranean , baba ganoush gluten free dinner , 1
mediterranean , bacon wrapped dates gluten free dinner , 1
mediterranean , banyan kabob gluten free dinner , 1
mediterranean , bar steak platter , 1
mediterranean , beef and lamb doner shawarma kebab , 1
mediterranean , black angus beef ribeye kabob wrap , 1
mediterranean , blackened tuna or salmon salad gluten free dinner , 1
mediterranean , borek , 1
mediterranean , bottled water , 1
turkish , a la turca lentil soup , 1
turkish , bar steak platter , 1
turkish , beef and lamb doner shawarma kebab , 1
turkish , borek , 1
turkish , cacik , 1
turkish , 

streetvendors , piece chicken wings , 1
streetvendors , taco , 1
streetvendors , tostadas , 1
cuban , bbq pork , 1
cuban , buffalo chicken , 1
cuban , cuban style black beans , 1
cuban , elote corn , 1
cuban , guava cream cheese , 1
cuban , nacho mamas nachos , 1
cuban , nachomale , 1
cuban , pineapple cream cheese , 1
cuban , slow roasted cuban pork , 1
cuban , three cheese , 1


In [113]:
#Dictionary of food that is shared across cuisines
shared_counts = {}

for key, value in shared_food.items():
    unique, counts = np.unique(value, return_counts=True)
    shared_counter = {}
    shared_counts[key] = {}
    for x in unique:
        shared_counter[x] = counts[np.where(unique==x)]

    popular_shares = sorted(shared_counter, key = shared_counter.get, reverse = True)
    top_shares = popular_shares[:20]
    for share in top_shares:
        shared_counts[key][share] = counts[np.where(unique==share)][0]
        print(key, ",", share, ",", shared_counts[key][share])
                
    

signature dry , chinese , 1
dry chili , chinese , 4
dry chili , szechuan , 2
dry chili , asianfusion , 1
chili fantail , chinese , 1
fantail shrimp , chinese , 2
fantail shrimp , bbq , 1
chicken egg , chinese , 11
chicken egg , cantonese , 4
chicken egg , dimsum , 3
chicken egg , seafood , 3
chicken egg , juicebars , 1
chicken egg , sushi , 1
chicken egg , szechuan , 1
chicken egg , thai , 1
egg roll , chinese , 16
egg roll , juicebars , 3
egg roll , bbq , 2
egg roll , dimsum , 2
egg roll , noodles , 2
egg roll , seafood , 2
egg roll , sushi , 2
egg roll , asianfusion , 1
egg roll , cantonese , 1
egg roll , panasian , 1
egg roll , thai , 1
egg roll , vietnamese , 1
roll 2pc , chinese , 1
philly cheese , pizza , 2
philly cheese , burgers , 1
philly cheese , chinese , 1
philly cheese , newamerican , 1
philly cheese , sandwiches , 1
cheese steak , pizza , 2
cheese steak , burgers , 1
cheese steak , chinese , 1
cheese steak , newamerican , 1
cheese steak , sandwiches , 1
steak spring , chi

chef special , dimsum , 1
chef special , foodstands , 1
chef special , seafood , 1
chef special , szechuan , 1
chef special , thai , 1
special chinese , chinese , 1
chinese wine , chinese , 1
wine and , chinese , 1
and soup , chinese , 1
and soup , dimsum , 1
and soup , juicebars , 1
and soup , seafood , 1
soup chicken , chinese , 1
chicken pot , chinese , 4
chicken pot , noodles , 1
chicken pot , szechuan , 1
mccb special , chinese , 1
special fried , chinese , 6
special fried , szechuan , 3
special fried , dimsum , 2
special fried , asianfusion , 1
special fried , cantonese , 1
special fried , seafood , 1
fried tofu , chinese , 11
fried tofu , taiwanese , 2
fried tofu , thai , 2
fried tofu , asianfusion , 1
fried tofu , bubbletea , 1
fried tofu , coffee , 1
fried tofu , foodstands , 1
fried tofu , noodles , 1
fried tofu , seafood , 1
fried tofu , shavedice , 1
fried tofu , vietnamese , 1
traditional sweet , chinese , 1
sour short , chinese , 1
short ribs , chinese , 11
short ribs , d

and cauliflower , chinese , 1
steak and , breakfast_brunch , 3
steak and , chinese , 2
steak and , mexican , 2
steak and , newamerican , 2
steak and , british , 1
steak and , burgers , 1
steak and , diners , 1
steak and , foodtrucks , 1
steak and , pubs , 1
steak and , salad , 1
steak and , sandwiches , 1
steak and , steak , 1
shrimp and , chinese , 19
shrimp and , dimsum , 6
shrimp and , cantonese , 4
shrimp and , seafood , 4
shrimp and , bbq , 3
shrimp and , asianfusion , 2
shrimp and , juicebars , 2
shrimp and , sushi , 2
shrimp and , bars , 1
shrimp and , breakfast_brunch , 1
shrimp and , coffee , 1
shrimp and , delis , 1
shrimp and , italian , 1
shrimp and , japanese , 1
shrimp and , newamerican , 1
shrimp and , noodles , 1
shrimp and , szechuan , 1
shrimp and , taiwanese , 1
shrimp and , vietnamese , 1
lamb and , chinese , 3
lamb and , bbq , 1
lamb and , breakfast_brunch , 1
lamb and , cantonese , 1
lamb and , dimsum , 1
lamb and , mediterranean , 1
lamb and , mideastern , 1
pork

and shrimp , cantonese , 1
and shrimp , indpak , 1
and shrimp , italian , 1
and shrimp , mexican , 1
and shrimp , panasian , 1
and shrimp , senegalese , 1
and shrimp , vietnamese , 1
sea whelk, , bbq , 1
sea whelk, , chinese , 1
whelk, leek , bbq , 1
whelk, leek , chinese , 1
leek and , bbq , 1
leek and , chinese , 1
shrimp, cucumber , bbq , 1
shrimp, cucumber , chinese , 1
tomato and , chinese , 4
tomato and , bbq , 1
tomato and , cantonese , 1
tomato and , dimsum , 1
tomato and , seafood , 1
tomato and , sushi , 1
tomato and , szechuan , 1
and celery , chinese , 4
and celery , bbq , 1
and celery , coffee , 1
and celery , taiwanese , 1
and green , chinese , 12
and green , bbq , 2
and green , cantonese , 2
and green , dimsum , 2
and green , seafood , 2
and green , szechuan , 2
and green , asianfusion , 1
and green , coffee , 1
and green , juicebars , 1
and green , sushi , 1
and green , taiwanese , 1
and green , tradamerican , 1
and mushroom , chinese , 5
and mushroom , asianfusion , 1


chicken in , dimsum , 1
chicken in , noodles , 1
chicken in , seafood , 1
chicken in , szechuan , 1
in black , chinese , 12
in black , seafood , 3
in black , asianfusion , 2
in black , cantonese , 2
in black , dimsum , 2
in black , coffee , 1
in black , noodles , 1
in black , taiwanese , 1
pepper sauce , chinese , 10
pepper sauce , seafood , 6
pepper sauce , cantonese , 5
pepper sauce , dimsum , 5
pepper sauce , szechuan , 3
pepper sauce , asianfusion , 2
pepper sauce , coffee , 1
pepper sauce , juicebars , 1
pepper sauce , taiwanese , 1
with ginger , chinese , 13
with ginger , dimsum , 4
with ginger , seafood , 3
with ginger , cantonese , 2
with ginger , asianfusion , 1
with ginger , coffee , 1
with ginger , noodles , 1
with ginger , szechuan , 1
with ginger , taiwanese , 1
ginger and , chinese , 13
ginger and , seafood , 6
ginger and , dimsum , 5
ginger and , cantonese , 3
ginger and , asianfusion , 2
ginger and , bbq , 1
ginger and , coffee , 1
ginger and , juicebars , 1
ginger and 

...

sesame dressing , cantonese , 1
sesame dressing , dimsum , 1
leaves scented , cantonese , 1
leaves scented , dimsum , 1
scented chicken, , cantonese , 1
scented chicken, , dimsum , 1
chicken, steamed , cantonese , 1
chicken, steamed , dimsum , 1
with cloud , cantonese , 1
with cloud , dimsum , 1
cloud wood , cantonese , 1
cloud wood , dimsum , 1
ear lily , cantonese , 1
ear lily , dimsum , 1
rural 5 , cantonese , 1
rural 5 , dimsum , 1
spices duck , cantonese , 1
spices duck , dimsum , 1
intestine sauteed , cantonese , 1
intestine sauteed , dimsum , 1
pig's intestine, , cantonese , 1
pig's intestine, , dimsum , 1
intestine, bitter , cantonese , 1
intestine, bitter , dimsum , 1
bitter melon, , cantonese , 1
bitter melon, , dimsum , 1
melon, sauteed , cantonese , 1
melon, sauteed , dimsum , 1
rustic stir , cantonese , 1
rustic stir , dimsum , 1
crispy golden , cantonese , 1
crispy golden , dimsum , 1
golden ribs , cantonese , 1
golden ribs , dimsum , 1
pineapple pork , cantonese , 1
pine

special smoked , chinese , 1
crispy rangoon , chinese , 1
bamboo soup , chinese , 1
flounder kow , chinese , 1
steamed oysters , chinese , 1
oysters with , chinese , 1
striped maws , chinese , 1
pepper baby , chinese , 1
baby codfish , chinese , 1
shrimp almond , chinese , 2
shrimp broccoli , chinese , 2
of bamboo , chinese , 1
glazed and , chinese , 1
and marinated , chinese , 1
marinated short , chinese , 1
walnut beef , chinese , 1
of mushrooms , chinese , 1
fresh seasonal , chinese , 1
poached sliced , chinese , 1
fried squab , chinese , 1
fried quail , chinese , 1
sour chciken , chinese , 1
mayflower egg , chinese , 1
vegetables egg , chinese , 1
hollow vegetable , chinese , 2
braised vegetables , chinese , 1
bean western , chinese , 1
water glass , chinese , 1
glass with , chinese , 1
with french , chinese , 1
french garlic , chinese , 1
chop sum , chinese , 1
mayflower fried , chinese , 1
rich fried , chinese , 1
fujian fried , chinese , 1
seafood pineapple , chinese , 1
ginger 

the northsider , breakfast_brunch , 1
the northsider , newamerican , 1
northsider burger , breakfast_brunch , 1
northsider burger , newamerican , 1
carb burger , breakfast_brunch , 1
carb burger , newamerican , 1
the bacon , breakfast_brunch , 1
the bacon , newamerican , 1
bacon me , breakfast_brunch , 1
bacon me , newamerican , 1
me crazy , breakfast_brunch , 1
me crazy , newamerican , 1
crazy burger , breakfast_brunch , 1
crazy burger , newamerican , 1
the plain , breakfast_brunch , 1
the plain , newamerican , 1
plain jane , breakfast_brunch , 1
plain jane , newamerican , 1
jane burger , breakfast_brunch , 1
jane burger , newamerican , 1
silver dollar , breakfast_brunch , 1
silver dollar , newamerican , 1
dollar pancakes , breakfast_brunch , 1
dollar pancakes , newamerican , 1
pancakes kids , breakfast_brunch , 1
pancakes kids , newamerican , 1
one egg , breakfast_brunch , 1
one egg , newamerican , 1
sausage kids , breakfast_brunch , 1
sausage kids , newamerican , 1
fingers kids , br

wings tray , pizza , 1
spaghetti pasta , pizza , 1
pasta tray , pizza , 1
lasagna pasta , pizza , 1
meat ravioli , pizza , 1
ravioli pasta , pizza , 1
sandwich tray , pizza , 1
club tray , pizza , 1
mediterranean veggie , pizza , 1
calzone tray , pizza , 1
alfredo calzone , pizza , 1
ranch chicken , pizza , 1
marinara chicken , pizza , 1
cheese calzone , pizza , 2
cheese calzone , italian , 1
ranch vegetarian , pizza , 1
vegetarian calzone , pizza , 1
pesto vegetarian , pizza , 1
marinara vegetarian , pizza , 1
cheesecake tray , pizza , 1
bulls tender , chicken_wings , 1
tender tip-off , chicken_wings , 1
cent boneless , chicken_wings , 1
boneless wings , chicken_wings , 1
boneless wing , chicken_wings , 1
wing lunch , chicken_wings , 1
small 6pc , chicken_wings , 1
wing combo , chicken_wings , 1
medium 8pc , chicken_wings , 1
large 10pc , chicken_wings , 1
small 3pc , chicken_wings , 1
wingstop crispy , chicken_wings , 1
crispy tender , chicken_wings , 1
tender combo , chicken_wings ,

chinatown chicken , breakfast_brunch , 1
chinatown chicken , burgers , 1
chinatown chicken , salad , 1
burger your , bars , 1
burger your , breakfast_brunch , 1
burger your , burgers , 1
burger your , salad , 1
burger your , tradamerican , 1
your way , bars , 1
your way , breakfast_brunch , 1
your way , burgers , 1
your way , salad , 1
your way , tradamerican , 1
pieces chicken , chicken_wings , 1
pieces chicken , hotdogs , 1
piece dinner , chicken_wings , 1
piece dinner , hotdogs , 1
piece tenders , chicken_wings , 1
piece tenders , hotdogs , 1
tenders dinner , chicken_wings , 1
tenders dinner , hotdogs , 1
lb popcorn , chicken_wings , 1
lb popcorn , hotdogs , 1
shrimp only , chicken_wings , 1
shrimp only , hotdogs , 1
cajun fish , chicken_wings , 1
cajun fish , hotdogs , 1
fish only , chicken_wings , 1
fish only , hotdogs , 1
pcs cajun , chicken_wings , 1
pcs cajun , hotdogs , 1
chicken po , chicken_wings , 1
chicken po , hotdogs , 1
po boy , chicken_wings , 1
po boy , hotdogs , 1
fi

In [100]:
### Bigrams - most common in each cuisine
for key, value in category_bigrams.items():
    unique, counts = np.unique(value, return_counts=True)
    bigram_counter = {}
    
    for x in unique:
        bigram_counter[x] = counts[np.where(unique==x)]
    
    popular_bigrams = sorted(bigram_counter, key = bigram_counter.get, reverse = True)
    top_bigrams = popular_bigrams[:50]
    for bigram in top_bigrams:
        print(key, "," , bigram, ",", counts[np.where(unique==bigram)][0])


bubbletea , milk tea , 69
bubbletea , noodle soup , 33
bubbletea , rice box , 27
bubbletea , braised pork , 21
bubbletea , over rice , 21
bubbletea , black tea , 18
bubbletea , egg waffle , 15
bubbletea , green tea , 15
bubbletea , pork over , 15
bubbletea , spicy beef , 15
bubbletea , winter melon , 15
bubbletea , beef noodle , 12
bubbletea , oolong tea , 12
bubbletea , tea mousse , 12
bubbletea , bowl of , 9
bubbletea , green milk , 9
bubbletea , ground pork , 9
bubbletea , noodle with , 9
bubbletea , passion fruit , 9
bubbletea , peach oolong , 9
bubbletea , shredded chicken , 9
bubbletea , square q , 9
bubbletea , stir fried , 9
bubbletea , tea latte , 9
bubbletea , tea with , 9
bubbletea , tendon noodle , 9
bubbletea , aiyu jelly , 6
bubbletea , and pepper , 6
bubbletea , and pork , 6
bubbletea , and tendon , 6
bubbletea , basil fried , 6
bubbletea , bbq pork , 6
bubbletea , bean curd , 6
bubbletea , beef and , 6
bubbletea , belly rice , 6
bubbletea , black forest , 6
bubbletea , 

mongolian , fried mini , 3
mongolian , fried rice , 3
mongolian , griddle cooked , 3
mongolian , grilled bun , 3
mongolian , grilled fish , 3
mongolian , herbal tea , 3
mongolian , hot and , 3
mongolian , iced tea , 3
mongolian , lamb kebab , 3
mongolian , lamb leg , 3
mongolian , lamb ribs , 3
mongolian , mini buns , 3
mongolian , mongolian meat , 3
mongolian , noodle salad , 3
mongolian , noodle soup , 3
mongolian , pork meat , 3
mongolian , pork sausage , 3
mongolian , potato slices , 3
mongolian , ribbon noodle , 3
mongolian , root noodles , 3
mongolian , scallion pancake , 3
mongolian , sesame flat , 3
mongolian , shredded spicy , 3
mongolian , sour fern , 3
mongolian , spicy noodles , 3
mongolian , spicy potato , 3
mongolian , squid body , 3
mongolian , sweet corn , 3
hotpot , chicken wings , 6
hotpot , meat pie , 6
hotpot , and sour , 3
hotpot , beef aorta , 3
hotpot , beef backstraps , 3
hotpot , beef noodle , 3
hotpot , black iced , 3
hotpot , chicken broth , 3
hotpot , chongq

british , large drip , 3
british , mac 'n' , 3
british , mushroom and , 3
british , o' pickles , 3
british , orange juice , 3
british , pickled egg , 3
british , pleasant farms , 3
british , ploughman's plate , 3
british , pork pie , 3
british , pot o' , 3
british , premium steak , 3
british , pub burger , 3
british , rarebit mac , 3
british , royal ginger , 3
british , scotch egg , 3
british , small drip , 3
british , soup and , 3
british , special pie , 3
british , steak and , 3
british , steak pasty , 3
british , wee pot , 3
british , welsh rarebit , 3
pubs , build your , 6
pubs , drip coffee , 6
pubs , your own , 6
pubs , 'n' cheese , 3
pubs , a blanket , 3
pubs , ala plancha , 3
pubs , ale pie , 3
pubs , anchors away , 3
pubs , and ale , 3
pubs , and bread , 3
pubs , and kale , 3
pubs , and mash , 3
pubs , and meatballs , 3
pubs , and pomegranate , 3
pubs , avocado rolls , 3
pubs , away fish , 3
pubs , bacon wrapped , 3
pubs , balti pie , 3
pubs , bangers and , 3
pubs , beef sandw

thai , mixed vegetables , 12
thai , pad ped , 12
thai , pot stickers , 12
thai , shrimp fried , 12
thai , shrimp with , 12
thai , thai iced , 12
thai , vegetable egg , 12
thai , with mixed , 12
thai , and sour , 9
thai , brown rice , 9
thai , chicken fried , 9
thai , chicken party , 9
thai , chicken plate , 9
thai , egg roll , 9
thai , fried tofu , 9
thai , garlic sauce , 9
thai , general tso's , 9
thai , hot and , 9
thai , iced coffee , 9
thai , iced tea , 9
thai , jumbo shrimp , 9
thai , kra praow , 9
thai , lard nar , 9
thai , orange chicken , 9
thai , pad kra , 9
thai , pad prik , 9
thai , seafood combination , 9
thai , sesame chicken , 9
thai , shrimp egg , 9
thai , spring rolls , 9
thai , tom yum , 9
thai , with garlic , 9
thai , woon sen , 9
thai , bami pad , 6
senegalese , lunch special , 18
senegalese , yassa chicken , 9
senegalese , yassa fish , 9
senegalese , curry with , 6
senegalese , dibi chicken , 6
senegalese , fish fillet , 6
senegalese , house thiebaut , 6
senegalese 

polish , scallion potato , 3
polish , serving of , 3
polish , to anything. , 3
sportsbars , & artichoke , 6
sportsbars , artichoke dip , 6
sportsbars , & giardinera , 3
sportsbars , & swiss , 3
sportsbars , a bun , 3
sportsbars , all american , 3
sportsbars , beef & , 3
sportsbars , beef polish , 3
sportsbars , beef sandwich , 3
sportsbars , beer (bottle) , 3
sportsbars , bottled water , 3
sportsbars , buffalo wings , 3
sportsbars , cheese dog , 3
sportsbars , cheesy beef , 3
sportsbars , chicago style , 3
sportsbars , chili cheese , 3
sportsbars , club soda , 3
sportsbars , diet rite , 3
sportsbars , dr. pepper , 3
sportsbars , garlic bread , 3
sportsbars , ginger ale , 3
sportsbars , goose island , 3
sportsbars , gravy bread , 3
sportsbars , hot dog , 3
sportsbars , hot giardeniara , 3
sportsbars , hot tamale , 3
sportsbars , island root , 3
sportsbars , italian beef , 3
sportsbars , italian cheesy , 3
sportsbars , italian poorboy , 3
sportsbars , jalapeno & , 3
sportsbars , jalapeno

foodstands , crab rangoon , 6
foodstands , egg rolls , 6
foodstands , green curry , 6
foodstands , pad curry , 6
foodstands , pad ginger , 6
foodstands , pad see , 6
foodstands , pad thai , 6
foodstands , see ewe , 6
foodstands , spicy basil , 6
foodstands , thai iced , 6
foodstands , thai panang , 6
foodstands , vegetable egg , 6
foodstands , a blanket , 3
foodstands , basil fried , 3
foodstands , beef plate , 3
foodstands , beef salad , 3
foodstands , bodhi vegetables , 3
foodstands , bottled water , 3
foodstands , brown rice , 3
foodstands , chef special , 3
foodstands , chicken soup , 3
foodstands , clay pot , 3
foodstands , crab fried , 3
foodstands , crispy fried , 3
foodstands , curry beef , 3
foodstands , curry fried , 3
foodstands , curry noodles , 3
foodstands , curry seafood , 3
foodstands , curry vegetable , 3
foodstands , fried tofu , 3
foodstands , garlic prawns , 3
foodstands , ginger tofu , 3
foodstands , ginger wings , 3
foodstands , grilled pork , 3
foodstands , herba

In [234]:
#Words - most common in each cuisine
for key, value in category_words.items():
    unique, counts = np.unique(value, return_counts=True)
    word_counter = {}
    
    for x in unique:
        word_counter[x] = counts[np.where(unique==x)]
    
    popular_words = sorted(word_counter, key = word_counter.get, reverse = True)
    top_words = popular_words[:50]
    for word in top_words:
        print(key, "," , word, ",", counts[np.where(unique==word)][0])
        
    #print(key)
    #print(unique, counts)

bubbletea , tea , 49
bubbletea , milk , 24
bubbletea , rice , 20
bubbletea , pork , 19
bubbletea , noodle , 15
bubbletea , braised , 13
bubbletea , chicken , 12
bubbletea , soup , 12
bubbletea , box , 10
bubbletea , beef , 8
bubbletea , black , 8
bubbletea , fried , 8
bubbletea , green , 8
bubbletea , mousse , 8
bubbletea , egg , 7
bubbletea , matcha , 7
bubbletea , oolong , 7
bubbletea , over , 7
bubbletea , crispy , 6
bubbletea , latte , 6
bubbletea , spicy , 6
bubbletea , strawberry , 6
bubbletea , basil , 5
bubbletea , chocolate , 5
bubbletea , jelly , 5
bubbletea , mango , 5
bubbletea , melon , 5
bubbletea , sauce , 5
bubbletea , smoothie , 5
bubbletea , tofu , 5
bubbletea , waffle , 5
bubbletea , winter , 5
bubbletea , yogurt , 5
bubbletea , bean , 4
bubbletea , fruit , 4
bubbletea , shredded , 4
bubbletea , snow , 4
bubbletea , taiwanese , 4
bubbletea , bbq , 3
bubbletea , bowl , 3
bubbletea , fresh , 3
bubbletea , ground , 3
bubbletea , honey , 3
bubbletea , lychee , 3
bubblete

caribbean , red , 5
caribbean , salad , 5
caribbean , shrimp , 4
caribbean , snapper , 4
caribbean , wings , 4
caribbean , curry , 3
caribbean , salmon , 3
caribbean , soda , 3
caribbean , tilapia , 3
caribbean , white , 3
caribbean , beans , 2
caribbean , brown , 2
caribbean , catfish , 2
caribbean , mix , 2
caribbean , mixed , 2
caribbean , pineapple , 2
caribbean , rice , 2
caribbean , steamed , 2
caribbean , stew , 2
caribbean , wrap , 2
caribbean , beef , 1
caribbean , beer , 1
caribbean , bread , 1
caribbean , broasted , 1
caribbean , cabbage , 1
caribbean , callaloo , 1
caribbean , can , 1
caribbean , candy , 1
caribbean , cesar , 1
caribbean , chardonnay , 1
caribbean , cilantro , 1
caribbean , coconut , 1
caribbean , combination , 1
caribbean , corn , 1
caribbean , cream , 1
caribbean , cuban , 1
caribbean , dark , 1
caribbean , diner , 1
caribbean , dinner , 1
caribbean , escovitch , 1
caribbean , excoveitched , 1
caribbean , extra , 1
caribbean , fry , 1
caribbean , garden ,

salad , kimchi , 2
salad , lamb , 2
salad , mac , 2
salad , onion , 2
salad , oscar , 2
ramen , rice , 6
ramen , kuro , 5
ramen , yakisoba , 5
ramen , bowl , 4
ramen , buta , 4
ramen , chazuke , 3
ramen , pork , 3
ramen , toppings , 3
ramen , turkey , 3
ramen , bone , 2
ramen , broth , 2
ramen , kamo , 2
ramen , sausage , 2
ramen , spicy , 2
ramen , (dipping , 1
ramen , (duck , 1
ramen , (pork) , 1
ramen , ajitsuke , 1
ramen , bacon , 1
ramen , beans , 1
ramen , beef , 1
ramen , belly , 1
ramen , bits , 1
ramen , bomb , 1
ramen , breast) , 1
ramen , broccoli , 1
ramen , buto , 1
ramen , cat , 1
ramen , chicken , 1
ramen , chocolates , 1
ramen , cider , 1
ramen , clam , 1
ramen , coppa , 1
ramen , crab , 1
ramen , da , 1
ramen , dango , 1
ramen , duck , 1
ramen , ear , 1
ramen , edamame , 1
ramen , garlic , 1
ramen , hot , 1
ramen , japanese , 1
ramen , loin , 1
ramen , lover , 1
ramen , meat , 1
ramen , minced , 1
ramen , moriawase , 1
ramen , noodle) , 1
ramen , oden , 1
ramen , oshin

mexican , ajo , 9
mexican , enchiladas , 9
mexican , flautas , 9
mexican , mexicana , 9
tapasmallplates , de , 16
tapasmallplates , con , 11
tapasmallplates , duck , 5
tapasmallplates , la , 5
tapasmallplates , pollo , 5
tapasmallplates , pork , 5
tapasmallplates , al , 4
tapasmallplates , chocolate , 4
tapasmallplates , feet , 4
tapasmallplates , plancha , 4
tapasmallplates , tarta , 4
tapasmallplates , chicken , 3
tapasmallplates , del , 3
tapasmallplates , handmade , 3
tapasmallplates , hot , 3
tapasmallplates , y , 3
tapasmallplates , berenjena , 2
tapasmallplates , buey , 2
tapasmallplates , cake , 2
tapasmallplates , calamares , 2
tapasmallplates , cazuelita , 2
tapasmallplates , gambas , 2
tapasmallplates , iced , 2
tapasmallplates , marinada , 2
tapasmallplates , mariscos , 2
tapasmallplates , mixed , 2
tapasmallplates , patatas , 2
tapasmallplates , pato , 2
tapasmallplates , pincho , 2
tapasmallplates , pot , 2
tapasmallplates , queso , 2
tapasmallplates , rice , 2
tapasmallp

comedyclubs , specialty , 1
comedyclubs , thompson , 1
comedyclubs , tramp , 1
comedyclubs , wine , 1
desserts , oz , 6
desserts , sauce , 4
desserts , filet , 3
desserts , grilled , 3
desserts , lobster , 3
desserts , rib , 3
desserts , spinach , 3
desserts , baby , 2
desserts , cfh , 2
desserts , cheese , 2
desserts , crab , 2
desserts , horseradish , 2
desserts , jumbo , 2
desserts , mac , 2
desserts , oscar , 2
desserts , oysters , 2
desserts , potato , 2
desserts , salad , 2
desserts , sautéed , 2
desserts , sea , 2
desserts , shrimp , 2
desserts , soup , 2
desserts , steak , 2
desserts , agnolotti , 1
desserts , all , 1
desserts , amish , 1
desserts , arugula , 1
desserts , asparagus , 1
desserts , au , 1
desserts , baked , 1
desserts , bass , 1
desserts , berkshire , 1
desserts , bisque , 1
desserts , bone-in , 1
desserts , bordelaise , 1
desserts , braised , 1
desserts , brick , 1
desserts , brussels , 1
desserts , burger , 1
desserts , béarnaise , 1
desserts , caesar , 1
desse

beergardens , strata , 1
beergardens , tomato , 1
beergardens , vegetable , 1
beergardens , yogurt , 1
sushi , roll , 182
sushi , chicken , 55
sushi , shrimp , 54
sushi , rice , 52
sushi , tuna , 46
sushi , soup , 44
sushi , spicy , 44
sushi , sashimi , 41
sushi , fried , 40
sushi , sushi , 36
sushi , beef , 32
sushi , salad , 31
sushi , salmon , 29
sushi , tempura , 26
sushi , plate , 22
sushi , white , 20
sushi , crab , 19
sushi , teriyaki , 19
sushi , noodle , 18
sushi , sauce , 17
sushi , seafood , 16
sushi , asparagus , 15
sushi , combination , 15
sushi , egg , 15
sushi , tofu , 15
sushi , hamachi , 14
sushi , sake , 14
sushi , tray , 14
sushi , dragon , 13
sushi , pepper , 13
sushi , pork , 13
sushi , avocado , 12
sushi , vegetable , 12
sushi , chow , 11
sushi , green , 11
sushi , maki , 11
sushi , smoothie , 11
sushi , sweet , 11
sushi , don , 10
sushi , ebi , 10
sushi , fun , 10
sushi , garlic , 10
sushi , grilled , 10
sushi , lobster , 10
sushi , party , 10
sushi , scallop , 1